In [7]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import csv
import re
import time

In [27]:
def scrape_page(url):
    response= requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        project_containers = soup.find_all('article', {'class': 'Box-row'}) # Encontrar os elementos de contêiner para os resultados
        
        data = []

        for container in project_containers: # Iterar sobre os resultados e extrair as informações
            # Procurar pelo título do projeto
            title_element = container.find('h2', {'class': 'h3'})
            title = title_element.text.strip() 
            # Remover espaços e quebras de linha extras no título
            title = ' '.join(title.split())

            # Procurar pela linguagem do projeto
            language_element = container.find('span', {'itemprop': 'programmingLanguage'})
            language = language_element.text.strip() if language_element else 'None'

            # Procurar pelo número de estrelas do projeto
            stars_element = container.find('a', {'class': 'Link Link--muted d-inline-block mr-3'})
            stars = stars_element.text.strip() if stars_element else 'None'

            # Procurar pelo número de estrelas hoje
            stars_element_today = container.find('span',{'class':'d-inline-block float-sm-right'})
            stars_today=stars_element_today.text.strip() if stars_element_today else 'None'
            match= re.search(r'^(\d+)', stars_today) # Use uma expressão regular para extrair apenas o número
            stars_today= int(match.group(1).replace(',','')) # obtendo o numero
        
            # Procurar forks do projeto
            forks_element=container.find_all('a',{'class':'Link Link--muted d-inline-block mr-3'}) # find_all pq estou procurando todas as class
            forks= forks_element[1].text.strip() if forks_element else 'None' # [1] estou selecionando o segundo elemento da class
        

            # Adicionar os dados à lista
            data.append([title, language, stars, stars_today, forks])
        return data
    else:
        print(f"Falha na requisição para {url} com status code: {response.status_code}")
        return None
    
def write_to_csv(data):
    with open('github.csv', 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile, delimiter=';')
        csv_writer.writerow(['ranking', 'project', 'language', 'stars', 'stars_today', 'forks'])
        
        for index, row in enumerate(data, start=1):
            csv_writer.writerow([index] + row)
# scraping de todas as paginas do site
if __name__ == "__main__":
    base_url = 'https://github.com/trending'
    num_pages = 5  # Defina o número de páginas que você deseja percorrer

    for page in range(1, num_pages + 1):
        url = f'{base_url}?page={page}'
        page_data = scrape_page(url)

        if page_data:
            write_to_csv(page_data)
        time.sleep(1)
    print("Dados extraídos com sucesso e gravados em 'github.csv'.")

 


Dados extraídos com sucesso e gravados em 'github.csv'.


In [28]:
# Ler o arquivo CSV substituindo vírgulas por pontos e ponto e vírgula por vírgula
df = pd.read_csv('github.csv', delimiter=';')

# Dividir a coluna "projeto" em duas colunas: "usuário" e "projeto"
df[['user', 'project']] = df['project'].str.split('/', n=1, expand=True)

# Substituir o ponto e vírgula por vírgula e a vírgula por ponto nas colunas numéricas
df[['stars', 'stars_today','forks']] = df[['stars', 'stars_today', 'forks']].replace({',': '.'}, regex=True)

# Reorganizar as colunas
df = df[['ranking', 'user', 'project', 'language', 'stars', 'stars_today', 'forks']]

# Salvar o DataFrame de volta como um novo arquivo CSV
df.to_csv('github.csv', index=False, sep=',', decimal='.')
print("Arquivo editado salvo como 'github.csv'.")


Arquivo editado salvo como 'github.csv'.


In [29]:
df= pd.read_csv('github.csv')
display(df)

,ranking,user,project,language,stars,stars_today,forks
0,1,Ryujinx,Ryujinx,C#,28.911,466,3.173
1,2,abi,screenshot-to-code,TypeScript,42.399,1,4.968
2,3,naver,dust3r,Python,2.787,380,303.000
3,4,vikhyat,moondream,Python,2.446,99,201.000
4,5,Loyalsoldier,clash-rules,NaN,15.058,108,1.267
5,6,dolphin-emu,dolphin,C++,11.741,50,2.501
6,7,huchenlei,ComfyUI-layerdiffuse,Python,520.000,184,59.000
7,8,atherosai,ui,HTML,2.557,164,494.000
8,9,yuzu-mirror,yuzu,C++,1.338,573,1.008
9,10,PabloMK7,citra,C++,584.000,148,308.000
